In [8]:
import numpy as np
import pandas as pd
import seaborn as sb
import io 
import requests
import datetime
import ondemand
from scipy.stats import pearsonr
from pandas.io.common import EmptyDataError
from requests.exceptions import HTTPError
import time
import os
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [9]:
#get business days
from pandas.tseries.offsets import *
daterange1 = pd.date_range('2017-12-01', '2019-01-20', freq=BDay())
daterange1 = list(daterange1.strftime('%Y-%m-%d'))
holidays = ['2017-12-25', '2018-01-01', '2018-01-15', '2018-02-19', '2018-03-30', '2018-05-28', '2018-07-04', '2018-09-03',\
            '2018-11-22', '2018-12-05', '2018-12-25', '2019-01-01']
for each in holidays:
    if each in daterange1:
        daterange1.remove(each)
daterange = daterange1[20:271] 

In [10]:
url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'

In [11]:
#for hold-1b day
all_res = pd.DataFrame([])
small_res = pd.DataFrame([])
mid_res = pd.DataFrame([])
large_res = pd.DataFrame([])
for cap in ['small', 'mid', 'large']:
    for date in daterange:
        for win in [1, 3]:#should include 1, 3
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                mydf = pd.read_csv('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/{}_{}_cap_window_{}_hold_1b_detail.csv'.\
                                   format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                continue
            if cap == 'small':
                small_res = pd.concat([mydf, small_res], sort = False, ignore_index = True)
            elif cap == 'mid':
                mid_res = pd.concat([mydf, mid_res], sort = False, ignore_index = True)
            elif cap == 'large':
                large_res = pd.concat([mydf, large_res], sort = False, ignore_index = True)
            #all_res = pd.concat([mydf, all_res], sort = False, ignore_index = True)

In [12]:
#for hold_1b
#for win-1
small_res_win1 = small_res.loc[small_res['window'] == 1]
mid_res_win1 = mid_res.loc[mid_res['window'] == 1]
large_res_win1 = large_res.loc[large_res['window'] == 1]
small_res_win1.to_csv('analysis/hold_1b_day/small_res_win1.csv', index = False)
mid_res_win1.to_csv('analysis/hold_1b_day/mid_res_win1.csv', index = False)
large_res_win1.to_csv('analysis/hold_1b_day/large_res_win1.csv', index = False)

#for win-3
small_res_win3 = small_res.loc[small_res['window'] == 3]
mid_res_win3 = mid_res.loc[mid_res['window'] == 3]
large_res_win3 = large_res.loc[large_res['window'] == 3]
small_res_win3.to_csv('analysis/hold_1b_day/small_res_win3.csv', index = False)
mid_res_win3.to_csv('analysis/hold_1b_day/mid_res_win3.csv', index = False)
large_res_win3.to_csv('analysis/hold_1b_day/large_res_win3.csv', index = False)

In [13]:
#for hold-1


In [66]:
all_res['actual_pageview_in_std'].describe()

count    1.504200e+04
mean              inf
std               NaN
min     -5.500000e-01
25%      2.500000e+00
50%      4.030000e+00
75%      6.350000e+00
max               inf
Name: actual_pageview_in_std, dtype: float64

In [67]:
all_res['pct_change(%)'].describe()

count    15042.000000
mean        -0.204810
std          6.030359
min        -84.883721
25%         -2.155376
50%         -0.133895
75%          1.681104
max        333.333333
Name: pct_change(%), dtype: float64

In [14]:
# fill the pct_change(%) group value for small_cap_window1_hold1b
#pageviewchange x < 2.5
for win in [1, 3]:#include 1, 3
    for cap in ['small', 'large', 'mid']:
        myres = pd.DataFrame([], index = [0, 1, 2, 3],\
                                                 columns = ['pageview_change_group', 'pct_change(%) < -1(% in samples(total))', \
                                                           '-1 <= pct_change(%) < 0(% in samples(total))', \
                                                            '0 <= pct_change(%) < 1(% in samples(total))', \
                                                            'pct_change(%) >= 1(% in samples(total))', 'samples(total)'])
        myres.iloc[0]['pageview_change_group'] = 'x < 2.5'
        myres.iloc[1]['pageview_change_group'] = '2.5 <= x < 4'
        myres.iloc[2]['pageview_change_group'] = '4 <= x < 6'
        myres.iloc[3]['pageview_change_group'] = 'x >= 6'
        rawdata = pd.read_csv('analysis/hold_1b_day/{}_res_win{}.csv'.format(cap, win))
        raw_0 = rawdata.loc[rawdata['actual_pageview_in_std'] < 2.5]
        raw_1 = rawdata.loc[(rawdata['actual_pageview_in_std'] < 4)&(rawdata['actual_pageview_in_std'] >= 2.5)]
        raw_2 = rawdata.loc[(rawdata['actual_pageview_in_std'] < 6)&(rawdata['actual_pageview_in_std'] >= 4)]
        raw_3 = rawdata.loc[rawdata['actual_pageview_in_std'] >= 6]
        myres.iloc[0, 5] = raw_0.shape[0]
        myres.iloc[1, 5] = raw_1.shape[0]
        myres.iloc[2, 5] = raw_2.shape[0]
        myres.iloc[3, 5] = raw_3.shape[0]
        raw = dict()
        raw[0] = raw_0
        raw[1] = raw_1
        raw[2] = raw_2
        raw[3] = raw_3
        for i in range(4):
            temp1 = raw[i].loc[raw[i]['pct_change(%)'] < -1]
            total = raw[i].shape[0] + 0.00001
            myres.iloc[i, 1] = (temp1.shape[0]/total)*100
            temp2 = raw[i].loc[(raw[i]['pct_change(%)'] >= -1)&(raw[i]['pct_change(%)'] < 0)]
            myres.iloc[i, 2] = (temp2.shape[0]/total)*100
            temp3 = raw[i].loc[(raw[i]['pct_change(%)'] >= 0)&(raw[i]['pct_change(%)'] < 1)]
            myres.iloc[i, 3] = (temp3.shape[0]/total)*100
            temp4 = raw[i].loc[raw[i]['pct_change(%)'] >= 1]
            myres.iloc[i, 4] = (temp4.shape[0]/total)*100
        if not os.path.exists(os.path.dirname('{}_cap/window_{}_day/hold_1b_day_statistics/'.format(cap, win))):
            os.makedirs(os.path.dirname('{}_cap/window_{}_day/hold_1b_day_statistics/'.format(cap, win)))
        myres.to_csv('{}_cap/window_{}_day/hold_1b_day_statistics/statistics_{}_cap_window_{}_hold_1b.csv'.format(cap, win, cap, win), index = False)
